#Chinese postman algorithm
###The Chinese Postman Problem is a mathematical problem in graph theory and combinatorial optimization. The goal is to find the shortest closed path or circuit that visits every edge of a connected, undirected graph at least once.
● If the graph has an Eulerian circuit, meaning a closed walk that covers every edge exactly once, then that circuit is the optimal solution.

● If the graph does not have an Eulerian circuit, then the problem becomes finding the smallest number of edges to duplicate (or the subset of edges with the minimum total weight) so that the resulting multigraph does have an Eulerian circuit.

This problem differs from the Traveling Salesman problem because the postman can repeat visited nodes and must visit every edge, whereas the salesman cannot repeat nodes and does not have to visit every edge. The Chinese Postman Problem can be solved in polynomial time, unlike the Traveling Salesman Problem, which is NP-hard.

In [1]:
import heapq #usage of this library reduces the computational cost of Dijkstra's algorithm
import numpy as np

In [14]:
#graphs on which we are going to test the algorithm
graph =                 [[0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0],
                       [4.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0],
                        [0.0, 8.0, 0.0, 7.0, 0.0, 4.0, 0.0, 0.0, 2.0],
                        [0.0, 0.0, 7.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 9.0, 0.0, 10.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 4.0, 0.0, 10.0, 0.0, 2.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 14.0, 0.0, 2.0, 0.0, 1.0, 6.0],
                        [8.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 7.0],
                        [0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 6.0, 7.0, 0.0]
                    ];

graph2 =                [[0.0, 3.0, 1.0, 0.0, 5.0, 0.0],
                        [3.0, 0.0, 0.0, 1.0, 0.0, 6.0],
                        [1.0, 0.0, 0.0, 0.0, 2.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
                        [5.0, 0.0, 2.0, 0.0, 0.0, 4.0],
                        [0.0, 6.0, 0.0, 1.0, 4.0, 0.0],

                    ];

graph3 = [[0.0,1.0,1.0,1.0,1.0],
          [1.0,0.0,1.0,0.0,0.0],
          [1.0,1.0,0.0,0.0,0.0],
          [1.0,0.0,0.0,0.0,1.0],
          [1.0,0.0,0.0,1.0,0.0]]



In [15]:
# Validate the graph for adjacency matrix properties and non-negative weights
def validate_graph(graph):
    """Validate the input graph."""
    if not all(len(row) == len(graph) for row in graph):
        raise ValueError("Graph must be a square adjacency matrix.")
    if not all(isinstance(weight, (int, float)) and weight >= 0 for row in graph for weight in row):
        raise ValueError("Graph must contain only non-negative weights.")

# Calculate the total weight of all edges in the graph
def sum_edges(graph):
    """Calculate the total weight of all edges."""
    return sum(graph[i][j] for i in range(len(graph)) for j in range(i, len(graph)) if graph[i][j] != float('inf'))

# Dijkstra's algorithm for finding shortest paths from a source node
def dijkstra(graph, source):
    """Find shortest paths from a source node to all other nodes using Dijkstra's algorithm.
    we need it to find the minimum weight perfect matching for pairs of vertices with odd degree"""
    n = len(graph)
    distances = [float('inf')] * n
    distances[source] = 0
    pq = [(0, source)]  # Priority queue as (distance, node)

    while pq:
        current_dist, current_node = heapq.heappop(pq) #removes from the queue the shortest distace node at each step
        if current_dist > distances[current_node]:
            continue
        for neighbor, weight in enumerate(graph[current_node]):
            if weight > 0 and weight != float('inf'):  # Edge exists
                distance = current_dist + weight #store the current best match
                if distance < distances[neighbor]:
                    distances[neighbor] = distance #if the new distance is smaller than previous, update the distanced variable
                    heapq.heappush(pq, (distance, neighbor))
    return distances

# Find all vertices with odd degree
def get_odd_vertices(graph):
    """Find all vertices with odd degree."""
    degrees = [sum(1 for weight in row if weight > 0 and weight != float('inf')) for row in graph]
    return [i for i, degree in enumerate(degrees) if degree % 2 != 0]

# Add edges to the graph to make it Eulerian
def add_edge(graph, u, v, weight):
    """Add an edge to the graph (make it symmetric for undirected graphs)."""
    graph[u][v] += weight
    graph[v][u] += weight

# Hungarian algorithm for finding the minimum weight perfect matching (find the optimal way to pair odd degree edges)
def hungarian_algorithm(cost_matrix):
    """Solve the assignment problem using the Hungarian algorithm."""
    from scipy.optimize import linear_sum_assignment
    cost_matrix = np.array(cost_matrix) #cost_matrix[i][j] represents the cost of pairing vertex i with vertex j, has to be complete
    row_ind, col_ind = linear_sum_assignment(cost_matrix) #this library find the optimal assignment for the cost matrix
    optimal_assignment = list(zip(row_ind, col_ind)) #stores the indices of optimal assignment
    minimum_cost = sum(cost_matrix[r, c] for r, c in optimal_assignment) #cost is calculated by summing the costs of all assigned pairs
    return optimal_assignment, minimum_cost

# Solve the Chinese Postman Problem
def chinese_postman_problem_with_hungarian(graph):
    """Solve the Chinese Postman Problem using the Hungarian algorithm."""
    validate_graph(graph)
    print("Original graph:")
    for row in graph:
        print(row)

    total_weight = sum_edges(graph)
    odd_vertices = get_odd_vertices(graph)
    print("\nOdd-degree vertices:", odd_vertices)

    if not odd_vertices:
        print("\nThe graph is already Eulerian.")
        return total_weight

    # Prepare the cost matrix for the Hungarian algorithm
    num_odd_vertices = len(odd_vertices)
    cost_matrix = [[float('inf')] * num_odd_vertices for _ in range(num_odd_vertices)]

    for i in range(num_odd_vertices):
        for j in range(num_odd_vertices):
            if i != j:
                # Use Dijkstra's algorithm to calculate shortest path between vertices
                distances = dijkstra(graph, odd_vertices[i])
                cost_matrix[i][j] = distances[odd_vertices[j]]

    # Convert the cost matrix to a numpy array
    cost_matrix = np.array(cost_matrix)

    # Apply the Hungarian algorithm
    assignment, added_weight = hungarian_algorithm(cost_matrix)
    print("\nMinimum weight matching:", [(odd_vertices[i], odd_vertices[j]) for i, j in assignment])
    print("Added weight:", added_weight)

    # Add the matching edges to make the graph Eulerian
    for i, j in assignment:
        add_edge(graph, odd_vertices[i], odd_vertices[j], cost_matrix[i][j])

    print("\nGraph after adding edges:")
    for row in graph:
        print(row)

    total_weight += added_weight
    print("\nTotal weight of the Chinese Postman Circuit:", total_weight)
    return total_weight


# Solve the problem
chinese_postman_problem_with_hungarian(graph)


Original graph:
[0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0]
[4.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0]
[0.0, 8.0, 0.0, 7.0, 0.0, 4.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 7.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 9.0, 0.0, 10.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 4.0, 0.0, 10.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 14.0, 0.0, 2.0, 0.0, 1.0, 6.0]
[8.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 7.0]
[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 6.0, 7.0, 0.0]

Odd-degree vertices: [1, 3, 5, 8]

Minimum weight matching: [(1, 8), (3, 5), (5, 3), (8, 1)]
Added weight: 42.0

Graph after adding edges:
[0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0]
[4.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 20.0]
[0.0, 8.0, 0.0, 7.0, 0.0, 4.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 7.0, 0.0, 9.0, 36.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 9.0, 0.0, 10.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 4.0, 22.0, 10.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 14.0, 0.0, 2.0, 0.0, 1.0, 6.0]
[8.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 7.0]
[0.0, 20.0, 2.0, 0.0, 0.0, 0.0, 6.0, 7.0, 0.0]



135.0

In [13]:
chinese_postman_problem_with_hungarian(graph2)

Original graph:
[0.0, 3.0, 1.0, 0.0, 5.0, 0.0]
[3.0, 0.0, 0.0, 1.0, 0.0, 6.0]
[1.0, 0.0, 0.0, 0.0, 2.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
[5.0, 0.0, 2.0, 0.0, 0.0, 4.0]
[0.0, 6.0, 0.0, 1.0, 4.0, 0.0]

Odd-degree vertices: [0, 1, 4, 5]

Minimum weight matching: [(0, 4), (1, 5), (4, 0), (5, 1)]
Added weight: 10.0

Graph after adding edges:
[0.0, 3.0, 1.0, 0.0, 11.0, 0.0]
[3.0, 0.0, 0.0, 1.0, 0.0, 10.0]
[1.0, 0.0, 0.0, 0.0, 2.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
[11.0, 0.0, 2.0, 0.0, 0.0, 4.0]
[0.0, 10.0, 0.0, 1.0, 4.0, 0.0]

Total weight of the Chinese Postman Circuit: 33.0


33.0

In [11]:
chinese_postman_problem_with_hungarian(graph3)

Original graph:
[0.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 1.0, 0.0]

Odd-degree vertices: []

The graph is already Eulerian.


6.0